In [ ]:
from pulsar_qcm.pulsar_qcm import pulsar_qcm
qcm = pulsar_qcm("qcm0", "192.168.0.3", debug=1)
qcm.get_idn()

In [ ]:
qcm.stop_sequencer()

In [ ]:
from quantify.scheduler.types import Schedule, Operation
from quantify.scheduler.gate_library import Rxy
from quantify.scheduler.pulse_library import SquarePulse
from quantify.scheduler.backends.pulsar_backend import pulsar_assembler_backend, configure_pulsar_sequencers
from quantify.scheduler.compilation import qcompile
from quantify.scheduler.backends.pulsar_backend import PulsarModulations
from quantify.scheduler.resources import QubitResource, CompositeResource, Pulsar_QCM_sequencer, Pulsar_QRM_sequencer

In [ ]:
DEVICE_TEST_CFG = {
    "qubits":
    {
        "q1": {"mw_amp180": 0.9, "mw_motzoi": 0.25, "mw_duration": 20e-9,
               "mw_modulation_freq": 200e6, "mw_ef_amp180": 0.87, "mw_ch": "qcm0.s0",
               "ro_ch": "qrm0.s0", "ro_pulse_amp": 0.5, "ro_pulse_modulation_freq": 0,
               "ro_pulse_type": "square", "ro_pulse_duration": 150e-9,
               "ro_acq_delay": 120e-9, "ro_acq_integration_time": 2500e-9,
               "ro_acq_weigth_type": "SSB",
               "init_duration": 250e-6
               }
    },
    "edges": {}
}

In [ ]:
qcm.stop_sequencer()

In [ ]:
import numpy as np

sched = Schedule('pulsar_params')
q0 = QubitResource('q1')

#sched.add(Rxy(180, 0, q0.name))
sched.add(SquarePulse(duration=40e-9, amp=0.6, ch='qcm0.s0'))

qcm0_s0 = Pulsar_QCM_sequencer('qcm0.s0', instrument_name='qcm0', seq_idx=0)
sched.add_resources([q0, qcm0_s0])

cfgs = qcompile(sched, DEVICE_TEST_CFG, backend=pulsar_assembler_backend, configure_hardware=True, debug=True)

In [ ]:
qcm.arm_sequencer()
qcm.start_sequencer()

In [ ]:
qcm.get_assembler_log()

In [ ]:
qcm.get_sequencer_state(0)

In [ ]:
from quantify.sequencer.backends.pulsar_backend import configure_pulsar_sequencers
qcm.reference_source(False)
qcm.stop_sequencer()
configure_pulsar_sequencers(cfgs)
qcm.arm_sequencer()
qcm.start_sequencer()